## Read JSON File

In [1]:
import json

# Open and read the JSON file
with open('C:/Users/Dell/Downloads/Modified_Dataset_Generation_JSON.json', 'r') as file:
    data = json.load(file)
# view the json file's key-value pairs
for key, value in data.items():
    print(f"{key}: {value}")

dataset_title: Dataset for Rabi Wheat in Bihar
feature_set: [{'feature_details': {'feature_name': 'Weighted Yield', 'feature_data_category': 'AREA_PRODUCTION_YIELD_ITEM', 'feature_data_location': 'STATE', 'feature_spatial_aggregation': 'DISTRICT_TO_STATE_SWA', 'feature_temporal_interval': 'SEASON', 'feature_temporal_aggregation': 'NA', 'is_target': 'true', 'temporal_interval_items': ''}}, {'feature_details': {'feature_name': 'TRF', 'feature_data_category': 'LOCATION_RAINFALL', 'feature_data_location': 'STATE', 'feature_spatial_aggregation': 'DISTRICT_TO_STATE_SWA', 'feature_temporal_interval': 'MONTH', 'feature_temporal_aggregation': 'DAILY_TO_MONTHLY_AVG', 'is_target': 'false', 'temporal_interval_items': [{'range_start': 'MAY', 'range_end': 'JUNE', 'base_year': 'SEASON_START'}, {'range_start': 'OCTOBER', 'range_end': 'DECEMBER', 'base_year': 'SEASON_START'}, {'range_start': 'FEBRUARY', 'range_end': 'FEBRUARY', 'base_year': 'SEASON_END'}]}}, {'feature_details': {'feature_name': 'AMXT',

## Month Range Extraction Functions

In [2]:
# Extract Range of Months for the columns
import datetime

def month_name_to_number(month_name):
    try:
        # Capitalize the first letter only, since strptime expects "January", not "JANUARY"
        dt = datetime.datetime.strptime(month_name.capitalize(), "%B")
        return dt.month
    except ValueError:
        return None
    
def get_months_between(start_num, end_num):
    if start_num <= end_num:
        return [(m) for m in range(start_num, end_num + 1)]
    else:
        return []


## Feature Extraction (Practice - 1)

In [3]:
feature_name = []
feature_data_category = []
month_range = []
for feature in data["feature_set"]:
    details = feature["feature_details"]
    feature_name.append(details["feature_name"])
    feature_data_category.append(details["feature_data_category"])
    range_pairs = []
    items = details.get("temporal_interval_items", [])
    if isinstance(items, list):
        for interval in items:
            start_num = month_name_to_number(interval["range_start"])
            #print(interval["range_start"])
            end_num = month_name_to_number(interval["range_end"])
            #print(end_num)
            range_pairs.extend(get_months_between(start_num,end_num))
    month_range.append(range_pairs)

## Import Python Librarires

In [4]:
from pypika import Query, Table, Field, functions as fn
from pypika.terms import Case
from pypika.enums import Order
from pypika.functions import Extract, Sum, Max, Avg, Date
import sys

## Weighted Yeild

In [5]:
# --- Subquery: final_aggregated ---
def apy_yeild(feature_data_category,feature_name,month_range,data):
    apy = Table(feature_data_category, schema="npcyf_schema")
    district = Table("DISTRICT", schema="npcyf_schema")
    state = Table("STATE", schema="npcyf_schema")
    crop = Table("CROP", schema="npcyf_schema")
    season = Table("SEASON", schema="npcyf_schema")
    sub_apy = (
        Query.from_(apy)
        .join(district).on(district.district_id == apy.location_id)
        .join(state).on(district.state_id == state.state_id)
        .join(crop).on(apy.crop_id == crop.crop_id)
        .join(season).on(apy.season_id == season.season_id)
        .select(
            state.state_name,
            district.district_name,
            apy.location_id,
            crop.crop_name,
            season.season_name,
            apy.apy_item_interval_start,
            apy.apy_data_collection_id,
            Max(Case().when(apy.apy_component == 'YIELD', apy.apy_item_value)).as_('yield'),
            Max(Case().when(apy.apy_component == 'CROPPED_AREA', apy.apy_item_value)).as_('cropped_area')
        )
        .where(
            (state.state_name == data['state'].lower()) &
            (season.season_name == data['season'].lower()) &
            (crop.crop_name == data['crop'].lower()) &
            (apy.apy_data_collection_id == data["apy_collection"]) &
            (apy.location_type == 'DISTRICT') &
            (apy.apy_item_interval_start >= int(data['start_year'])) &
            (apy.apy_item_interval_start <= int(data['end_year']))

        )
        .groupby(
            state.state_name,
            district.district_name,
            apy.location_id,
            crop.crop_name,
            season.season_name,
            apy.apy_item_interval_start,
            apy.apy_data_collection_id
    ))
    final_aggr = Table("final_aggregated")

    return (
        Query.from_(sub_apy.as_('final_aggregated'))
        .select(
            final_aggr.state_name,
            final_aggr.crop_name,
            final_aggr.season_name,
            final_aggr.apy_item_interval_start.as_('year'),
            Case()
            .when(Sum(final_aggr.cropped_area) != 0,
                fn.Sum(final_aggr['yield'] * final_aggr.cropped_area) / Sum(final_aggr.cropped_area))
            .else_(None)
            .as_(feature_name)
        )
        .groupby(
            final_aggr.state_name,
            final_aggr.crop_name,
            final_aggr.season_name,
            final_aggr.apy_item_interval_start,
            final_aggr.apy_data_collection_id
    )).as_('g')


## Max Rainfall

In [6]:
def max_rainfall(feature_data_category,alias_prefix,month_range,data):
    rain = Table(feature_data_category, schema="npcyf_schema")
    district = Table("DISTRICT", schema="npcyf_schema")
    state = Table("STATE", schema="npcyf_schema")
    rain_data = (
        Query.from_(rain)
        .join(district).on(rain.location_id == district.district_id)
        .join(state).on(district.state_id == state.state_id)
        .select(
            state.state_name,
            Extract('year', rain.rainfall_recorded_date).as_('year'),
            Extract('month', rain.rainfall_recorded_date).as_('month'),
            Extract('day', rain.rainfall_recorded_date).as_('day'),
            rain.rainfall_value
        )
        .where(
            (state.state_name == data['state'].lower()) &
            (rain.location_type == 'DISTRICT') &
            (rain.rainfall_data_collection_id == data["rainfall_collection"]) &
            (rain.rainfall_recorded_year >= data["start_year"])&
            (rain.rainfall_recorded_year <= data["end_year"]) &
            Extract('month', rain.rainfall_recorded_date).isin(range_pairs)
        )
    ).as_('rainfall_data')

    rainfall_expressions = []
    for month in month_range:
        label = f"{alias_prefix}_{month}"
        condition1 = (rain_data.month == month)
        expr = Sum(Case().when(condition1, rain_data.rainfall_value).else_(0)).as_(label)
        rainfall_expressions.append(expr)

    # Build final query
    return (
        Query.from_(rain_data)
        .select(
            rain_data.state_name,
            rain_data.year,
            *rainfall_expressions
        )
        .groupby(rain_data.state_name, rain_data.year)
    )

## Average Monthly Max Temperature

In [7]:
def avg_max_temperature(feature_data_category, alias_prefix,month_range,data):
    temp = Table(feature_data_category, schema="npcyf_schema")
    district = Table("DISTRICT", schema="npcyf_schema")
    state = Table("STATE", schema="npcyf_schema")
    sub_temp = (
        Query.from_(temp)
        .join(district).on(temp.location_id == district.district_id)
        .join(state).on(district.state_id == state.state_id)
        .select(
            state.state_name,
            Date(temp.temperature_recorded_date).as_('date'),
            Extract('year', temp.temperature_recorded_date).as_('year'),
            Extract('month', temp.temperature_recorded_date).as_('month'),
            Extract('day', temp.temperature_recorded_date).as_('day'),
            Avg(temp.temperature_value).as_('avg_temp_per_day')
        )
        .where(
            (temp.location_type == 'DISTRICT') &
            (state.state_name == data["state"].lower()) &
            (temp.temperature_data_collection_id == int(data['temperature_collection'])) &
            (temp.temperature_recorded_year >= data["start_year"])&
            (temp.temperature_recorded_year <= data["end_year"]) &
            Extract('month', temp.temperature_recorded_date).isin(month_range)
        )
        .groupby(state.state_name, Date(temp.temperature_recorded_date))
    ).as_(f'daily_state_temp_{alias_prefix}')

    temperate_expressions = []
    for month in month_range:
        label = f"{alias_prefix}_{month}"
        condition = (sub_temp.month == month)
        expr = Avg(Case().when(condition, sub_temp.avg_temp_per_day)).as_(label)
        temperate_expressions.append(expr)
    return (
        Query.from_(sub_temp)
        .select(
            sub_temp.state_name,
            sub_temp.year,
            *temperate_expressions
        ).groupby(sub_temp.state_name, sub_temp.year)
    )


## Unkown Category

In [8]:
def handle_default(feature_data_category,feature_name,month_range,data):
    print(f"Unknown feature category: {feature_name}")
    return 

## Case Function for Selecting Features

In [9]:
output = {}
category_handlers = {
    "AREA_PRODUCTION_YIELD_ITEM": apy_yeild,
    "LOCATION_RAINFALL": max_rainfall,
    "LOCATION_TEMPERATURE": avg_max_temperature
}
for feature in data["feature_set"]:
    details = feature["feature_details"]
    feature_name = details["feature_name"]
    feature_data_category = details["feature_data_category"]
    range_pairs = []
    items = details.get("temporal_interval_items", [])
    if isinstance(items, list):
        for interval in items:
            start_num = month_name_to_number(interval["range_start"])
            #print(interval["range_start"])
            end_num = month_name_to_number(interval["range_end"])
            #print(end_num)
            range_pairs.extend(get_months_between(start_num,end_num))
    month_range.append(range_pairs)
    handler = category_handlers.get(details["feature_data_category"], handle_default)
    if handler is not handle_default:
        output[feature_name] = handler(feature_data_category, feature_name, range_pairs, data)

In [10]:
for name, result in output.items():
    print(f"{result}")

SELECT "final_aggregated"."state_name","final_aggregated"."crop_name","final_aggregated"."season_name","final_aggregated"."apy_item_interval_start" "year",CASE WHEN SUM("final_aggregated"."cropped_area")<>0 THEN SUM("final_aggregated"."yield"*"final_aggregated"."cropped_area")/SUM("final_aggregated"."cropped_area") ELSE NULL END "Weighted Yield" FROM (SELECT "STATE"."state_name","DISTRICT"."district_name","AREA_PRODUCTION_YIELD_ITEM"."location_id","CROP"."crop_name","SEASON"."season_name","AREA_PRODUCTION_YIELD_ITEM"."apy_item_interval_start","AREA_PRODUCTION_YIELD_ITEM"."apy_data_collection_id",MAX(CASE WHEN "AREA_PRODUCTION_YIELD_ITEM"."apy_component"='YIELD' THEN "AREA_PRODUCTION_YIELD_ITEM"."apy_item_value" END) "yield",MAX(CASE WHEN "AREA_PRODUCTION_YIELD_ITEM"."apy_component"='CROPPED_AREA' THEN "AREA_PRODUCTION_YIELD_ITEM"."apy_item_value" END) "cropped_area" FROM "npcyf_schema"."AREA_PRODUCTION_YIELD_ITEM" JOIN "npcyf_schema"."DISTRICT" ON "DISTRICT"."district_id"="AREA_PRODUCT

## Final Join and Generate the Full Query

In [11]:
# Step 1: Generate table aliases
feature_names = list(output.keys())
tables = [Table(alias) for alias in feature_names]
tables
if not tables:
    raise RuntimeError("Stopping execution: condition failed.")



In [12]:
q = Query.from_(tables[0])

# Step 3: Join the remaining tables on state_name and year
for t in tables[1:]:
    q = q.join(t).on(
        (tables[0].state_name == t.state_name) &
        (tables[0].year == t.year)
    )

# Step 4: Select all columns (or specify if needed)
for t in tables:
    q = q.select(*(t.star,))  # selects all columns from each table

# Step 5: Get SQL (just the SELECT... FROM... JOIN part)
main_join_sql = q.get_sql()

# Step 6: Wrap the original subqueries as CTEs
cte_sql = "WITH\n" + ",\n".join(
    f'"{table_name}" AS (\n{output[table_name]}\n)' for table_name  in feature_names
)

# Step 7: Final full SQL
full_sql = f"{cte_sql}\n{main_join_sql}"


#### Save the SQL query as sql file

In [13]:
# Output the final query
print(full_sql)
with open("Generated_SQL.sql", "w") as file:
    file.write(full_sql)

WITH
"Weighted Yield" AS (
SELECT "final_aggregated"."state_name","final_aggregated"."crop_name","final_aggregated"."season_name","final_aggregated"."apy_item_interval_start" "year",CASE WHEN SUM("final_aggregated"."cropped_area")<>0 THEN SUM("final_aggregated"."yield"*"final_aggregated"."cropped_area")/SUM("final_aggregated"."cropped_area") ELSE NULL END "Weighted Yield" FROM (SELECT "STATE"."state_name","DISTRICT"."district_name","AREA_PRODUCTION_YIELD_ITEM"."location_id","CROP"."crop_name","SEASON"."season_name","AREA_PRODUCTION_YIELD_ITEM"."apy_item_interval_start","AREA_PRODUCTION_YIELD_ITEM"."apy_data_collection_id",MAX(CASE WHEN "AREA_PRODUCTION_YIELD_ITEM"."apy_component"='YIELD' THEN "AREA_PRODUCTION_YIELD_ITEM"."apy_item_value" END) "yield",MAX(CASE WHEN "AREA_PRODUCTION_YIELD_ITEM"."apy_component"='CROPPED_AREA' THEN "AREA_PRODUCTION_YIELD_ITEM"."apy_item_value" END) "cropped_area" FROM "npcyf_schema"."AREA_PRODUCTION_YIELD_ITEM" JOIN "npcyf_schema"."DISTRICT" ON "DISTRICT".

## Execute the SQL Script

In [14]:
# --- Execute SQL and print results ---
# Replace with your PostgreSQL credentials
import psycopg2
import pandas as pd
conn = psycopg2.connect(
        host="localhost",
        port="5433",
        dbname="New_Rainfall",
        user="postgres",
        password="vombolD007@"
    )

cursor = conn.cursor()
cursor.execute(full_sql)
columns = [desc[0] for desc in cursor.description]
rows = cursor.fetchall()
seen = {}
unique_data = []
for i, col in enumerate(columns):
    if col not in seen:
        seen[col] = i  # store the first occurrence index

# Keep only unique columns
unique_columns = list(seen.keys())
unique_indices = list(seen.values())

# Filter rows to keep only values for unique columns
filtered_rows = [[row[i] for i in unique_indices] for row in rows]

cursor.close()
conn.close()

## Save the result in a csv file

In [15]:
import numpy as np
import pandas as pd

# Combine rows' of value and columns' name and make a dataframe
df = pd.DataFrame(filtered_rows, columns=unique_columns)

# step1: convert all possible columns to numperic_type and leave rest columns as null
df_numeric = df.apply(pd.to_numeric, errors='coerce')

# Step 2: Round up numeric values to 2 decimal places
df_rounded = np.ceil(df_numeric * 100) / 100
df_rounded['year'] = df_rounded['year'].astype(int)

# Step 3: Fill back non-numeric (original) columns
df_final = df_rounded.combine_first(df)

# step 4: save the dataset in a csv file
df_final.to_csv('example_Function_Query_Output.csv')
df_final

,state_name,crop_name,season_name,year,Weighted Yield,TRF_5,TRF_6,TRF_10,TRF_11,TRF_12,TRF_2,AMXT_5,AMXT_6,AMXT_7
0,bihar,wheat,rabi,2000,2.15,4222.59,11335.75,673.3,12.85,0.59,388.36,35.63,33.67,32.62
1,bihar,wheat,rabi,2001,2.07,4764.34,9227.35,8166.88,64.08,0.0,95.39,35.48,32.89,32.39
2,bihar,wheat,rabi,2002,1.9,3235.19,5174.78,1580.61,95.84,25.89,443.28,35.48,34.87,33.9
3,bihar,wheat,rabi,2003,1.78,2361.18,9753.71,6416.59,6.3,154.62,1576.19,36.91,35.41,33.11
4,bihar,wheat,rabi,2004,1.61,2883.82,9945.22,2787.26,266.11,4.0,30.04,37.13,34.06,32.74
5,bihar,wheat,rabi,2005,1.41,1439.18,4106.28,2044.19,85.85,20.02,443.86,37.25,38.39,32.53
6,bihar,wheat,rabi,2006,2.01,2754.2,7418.72,933.93,78.32,74.6,145.09,35.89,34.72,32.92
7,bihar,wheat,rabi,2007,2.34,2739.76,6520.37,1603.47,659.08,556.1,1058.67,36.53,35.22,32.07
8,bihar,wheat,rabi,2008,2.05,2142.08,11159.63,1412.0,39.76,0.99,146.53,36.71,33.18,32.8
9,bihar,wheat,rabi,2009,2.09,4052.03,2977.36,3946.86,160.87,20.63,17.49,35.83,37.59,33.71
